In [ ]:
import os

from_drive = True  # same flag you already use

if os.environ.get("ATLAS_BOOTSTRAPPED") != "1":
    # ---------- GIT ON COLAB ONLY ----------
    try:
        from google.colab import userdata

        git_token = userdata.get('gitToken')
        git_user = userdata.get('gitUser')
        git_url = f'https://{git_token}@github.com/rene-aum/Atlas.git'
        branch_to_pull = 'dev'

        # Always start from /content for predictable paths
        os.chdir('/content')

        # Clone only if not already present
        if not os.path.isdir('Atlas'):
            !git clone {git_url}

        %cd Atlas

        # Make sure we are on the right branch and up to date
        !git fetch origin {branch_to_pull}
        !git checkout {branch_to_pull}
        !git pull origin {branch_to_pull}

        # Install dependencies once
        !pip install -r PipelinesConsumo/src/requirements.txt

        # Move into PipelinesConsumo, where your notebooks live
        %cd PipelinesConsumo

    except Exception as e:
        print(e)
        print('Running in other environment not colab probably!')

    # ---------- DRIVE + SHEETS ----------
    if from_drive:
        from pydrive2.auth import GoogleAuth
        from pydrive2.drive import GoogleDrive
        from google.colab import auth
        from oauth2client.client import GoogleCredentials
        import gspread
        from google.auth import default
        from gspread_dataframe import set_with_dataframe
        import gdown

        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        drive = GoogleDrive(gauth)

        creds, _ = default()
        gc = gspread.authorize(creds)

    # mark bootstrap as done for this kernel
    os.environ["ATLAS_BOOTSTRAPPED"] = "1"
else:
    print("Bootstrap already done in this kernel. Skipping setup.")


In [ ]:
from IPython import get_ipython

def run_notebook(path):
    print(f"\n=== Running {path} ===")
    get_ipython().run_line_magic("run", path)
    print(f"=== Finished {path} ===\n")

notebooks = [
    "../Orquestador/Testing/notebooks/nb1.ipynb",
    "../Orquestador/Testing/notebooks/nb2.ipynb",
]

for nb in notebooks:
    try:
        run_notebook(nb)
    except Exception as e:
        print(f"❌ {nb} failed: {e}")
        # decide if you want to break or continue
        break
